In [ ]:
import numpy as np
import pandas as pd
import statistics as st
import matplotlib.pyplot as plt
import math

from opentisim.containers import container_objects
from opentisim.containers import container_defaults
from opentisim.containers import container_system

from opentisim.core import core
from opentisim.plot import plot

plt.style.use('seaborn-darkgrid')

# 0. Prepare inputs

In [ ]:
startyear = 2021
lifecycle = 10

## 0.1 Generate demand forecast scenario

In [ ]:
years = list(range(startyear, startyear+lifecycle))

demand = []
for year in years:
    if year <= 2025:
        demand.append(300_000)
    else:
        demand.append(750_000)

scenario_data={'year': years, 'volume': demand}

# ladens
container_defaults.container_data['historic_data'] = []
container = container_objects.Commodity(**container_defaults.container_data)
container.scenario_data = pd.DataFrame(data=scenario_data)

# combine
demand = [container]


# plot scenarios 
if False:
    laden.plot_demand()

## 0.2 Generate vessels

In [ ]:
# instantiate vessels
fully_cellular = container_objects.Vessel(**container_defaults.fully_cellular_data)
panamax = container_objects.Vessel(**container_defaults.panamax_data)
panamax_max = container_objects.Vessel(**container_defaults.panamax_max_data)
post_panamax_I = container_objects.Vessel(**container_defaults.post_panamax_I_data)
post_panamax_II = container_objects.Vessel(**container_defaults.post_panamax_II_data)
new_panamax_data = container_objects.Vessel(**container_defaults.new_panamax_data)
VLCS = container_objects.Vessel(**container_defaults.VLCS_data)
ULCS = container_objects.Vessel(**container_defaults.ULCS_data)

vessels = [fully_cellular, panamax, panamax_max, post_panamax_I, post_panamax_II, new_panamax_data, VLCS, ULCS] 

# 1. Instatiate terminal system

In [ ]:
# Terminal_calcs=[]
# stack_cranes = ['rtg', 'rmg', 'sc', 'rs']
# # Cranes = [container_defaults.rmg', 'rtg', 'sc', 'rs']
# for crane in stack_cranes:   
#     Terminal_dummy = container_system.System(startyear = startyear, 
#                                              lifecycle = lifecycle, 
#                                              elements = demand + vessels, 
#                                              operational_hours = 8500,   
#                                              debug=False,
#                                              stack_equipment = crane,
#                                              laden_stack = crane) 
#     NPV, data=Terminal_dummy.simulate()
#     Terminal_calcs.append(data)
# print(Terminal_calcs)


In [ ]:
# def chunkIt(seq, num):
#     avg = len(seq) / float(num)
#     out = []
#     last = 0.0

#     while last < len(seq):
#         out.append(seq[int(last):int(last + avg)])
#         last += avg

#     return out

In [ ]:
# Terminal_calcs=[]
# data_overview = []
# stack_cranes = ['rtg', 'rmg', 'sc', 'rs']
# fuel_costs = [0.1, 0.5, 1, 2]
# power_costs = [0.0, 0.05, 0.1, 0.32]
# land_costs = [0, 50, 100, 375]


# for crane in stack_cranes:
#     for i in power_costs: 
#         Terminal_dummy = container_system.System(startyear = startyear, 
#                                                  lifecycle = lifecycle, 
#                                                  elements = demand + vessels, 
#                                                  operational_hours = 7500,   
#                                                  debug=False,
#                                                  stack_equipment = crane,
#                                                  laden_stack = crane,
#                                                  energy_price = i ) 
#         NPV, data=Terminal_dummy.simulate()
#         Terminal_calcs.append(NPV)
#         data_overview.append(data)
        
# print(Terminal_calcs)
# print(data_overview)


In [ ]:
# out=chunkIt(Terminal_calcs, len(land_costs))

# rtg=out[0]
# rmg=out[1]
# sc=out[2]
# rs=out[3]

# # rtg=[-205500000, -248300000, -304000000, -368240000]
# # rmg=[-234270000, -272064000, -321630000, -378327000]
# # sc=[-241610000, -281480000, -333630000, -393442000]
# # rs=[-217999000, -264414772, -324757000, -393380000]

# ind = np.arange(4)  # the x locations for the groups
# width = 0.1       # the width of the bars


# fig, ax = plt.subplots(figsize=(16, 7))
# ax.bar(ind, rtg, width = width, label ="RTG", color = 'coral')
# ax.bar(ind+width, rmg, width = width, label="RMG", color = 'darkcyan')
# ax.bar(ind+2*width, sc, width = width, label="Straddle carrier", color = 'darkkhaki')
# ax.bar(ind+3*width, rs, width = width, label="Reach stacker", color = 'cornflowerblue')
# ax.set_xticks(ind+3*width/3)
# ax.set_xticklabels(('20', '40', '70', '100'))
# plt.ylabel('Net present value [00 M USD]')
# plt.xlabel('Cost of labour [k USD/FTE]')
# plt.legend()
# ax.set_title('NPV per equipment for varying cost of labour')
# plt.show()
# plt.savefig('C:/Users/908538/Google Drive/Afstuderen/02. Model/Masterfile/Plots/fuel.png', dpi=400)


In [ ]:
Terminal = container_system.System(startyear = startyear, 
                         lifecycle = lifecycle, 
                         elements = demand + vessels, 
                         operational_hours = 7500,  
                         debug=True,
                         crane_type_defaults=container_defaults.sts_crane_data) 
Terminal.modelframe = list(range(startyear, startyear + lifecycle))

# 2. Start simulation
Develop terminal given the throughput scenario and investment triggers implemented

In [ ]:
Terminal.simulate()

## Report all elements

In [ ]:
if True: 
    for element in Terminal.elements:
        print("")
        print(element.name)
        print("")
        print(element.__dict__)

In [ ]:
cash_flows, cash_flows_WACC_real = core.add_cashflow_elements(Terminal)

In [ ]:
Terminal.opex_plot(cash_flows)

In [ ]:
Terminal.terminal_elements_plot()

In [ ]:
Terminal.land_use_plot()

In [ ]:
plot.cashflow_plot(cash_flows)

## SandBox

In [ ]:
core.NPV(Terminal)